<img src="../assets/header_notebook.png" />
<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>ESA - Black Sea Deoxygenation Emulator</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
# ----------
# Librairies
# ----------
import os
import sys
import cv2
import dawgz
import wandb
import xarray
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim

# Dawgz (jobs //)
from dawgz import job, schedule

# -------------------
# Librairies (Custom)
# -------------------
# Adding path to source folder to load custom modules
sys.path.append('/src')
sys.path.append('/src/debs/')
sys.path.insert(1, '/src/debs/')
sys.path.insert(1, '/scripts/')

# Moving to the .py directory
%cd src/debs/

## Loading libraries
from metrics     import *
from dataset     import *
from dataloader  import *
from tools       import *
from losses      import *

# -------
# Jupyter
# -------
%matplotlib inline
plt.rcParams.update({'font.size': 13})

# Making sure modules are reloaded when modified
%reload_ext autoreload
%autoreload 2

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Scripts</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
# Generating the preprocessed data
%run __generate.py

In [ ]:
# Generating the distributions of the data each month
%run __distributions.py

In [ ]:
# Training a neural network:
%run __training.py --config local

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Data</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
# Loading the different inputs
BSD_dataset = BlackSea_Dataset(year_start  = 2020,
                               year_end    = 2021,
                               month_start = 1,
                               month_end   = 12)

data_temperature   = BSD_dataset.get_data(variable = "temperature")
data_salinity      = BSD_dataset.get_data(variable = "salinity")
data_chlorophyll   = BSD_dataset.get_data(variable = "chlorophyll")
data_kshort        = BSD_dataset.get_data(variable = "kshort")
data_klong         = BSD_dataset.get_data(variable = "klong")
data_oxygen        = BSD_dataset.get_data(variable = "oxygen")
mask               = BSD_dataset.get_mask(False)
maskCS             = BSD_dataset.get_mask(True)

def generate_animation(data : np.array, mask : np.array, title : str, limits : list = [0, 1]):
    """Generate an animation of the data, i.e. used to inspect the data"""
    import os
    import cv2
    import imageio
    import numpy as np
    import matplotlib.pyplot as plt
    from PIL import Image
    from matplotlib.patches import Rectangle
    from datetime import datetime, timedelta

    # Converting to booleans for easier plot
    mask = mask > 0.5

    # Displaying information over terminal
    print(f"Data shape: {data.shape}")

    def index_to_date(i):
        """Used to convert an index to a date, i.e. useful for the title of the plot"""

        # Define the start date
        start_date = datetime(2020, 1, 1)

        # Calculate the offset from the start date
        delta = timedelta(days=i)

        # Calculate the corresponding date
        result_date = start_date + delta

        # Format the date as a string in the "YYYY-MM-DD" format
        return result_date.strftime('%Y-%m-%d')

    # Creation of a folder if it does not exist
    if not os.path.exists(f'../../analysis/images/{title}'):
        os.mkdir(f'../../analysis/images/{title}/')

    # Displaying information over terminal
    print(f"Generating images for {title}...")

    # Creating of the plots
    for i in range(data.shape[0]):
        plt.figure(figsize = (12, 8))
        plt.imshow(data[i], cmap='viridis', vmin = limits[0], vmax = limits[1])
        plt.colorbar(fraction = 0.021)
        plt.imshow(mask, cmap='gray', alpha=0.025)
        date_string = index_to_date(i)
        ax = plt.gca()
        box = Rectangle((0.85, 0.9), 0.46, 0.16, edgecolor='black', facecolor='black', transform=ax.transAxes)
        plt.annotate(date_string, xy=(0.98, 0.96), xycoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    fontsize=12, color='white')
        ax.add_patch(box)
        plt.grid(True, alpha=0.1)
        plt.savefig(f'../../analysis/images/{title}/{title}_{i}.png')
        plt.close()

    # Displaying information over terminal
    print("Loading images...")

    # Creation of the paths and opening the plots
    paths = [f"../../analysis/images/{title}/{title}_{i}.png" for i in range(data.shape[0])]
    image_array = []
    for my_file in paths:
        image = Image.open(my_file)
        image_array.append(image)

    # Displaying information over terminal
    print("Generating the video...")

    # Generating the video
    with imageio.get_writer(f'../../analysis/images/{title}.gif', mode='I') as writer:
        for filename in paths:
            image = imageio.imread(filename)
            writer.append_data(image)

# Generating the animations
generate_animation(data_oxygen,      maskCS,      "oxygen", limits = [0, 1])
generate_animation(data_temperature,   mask, "temperature", limits = [0, 1])
generate_animation(data_salinity,      mask,    "salinity", limits = [0, 1])
generate_animation(data_chlorophyll,   mask, "chlorophyll", limits = [0, 0.25])
generate_animation(data_kshort,        mask,      "kshort", limits = [0, 0.1])
generate_animation(data_klong,         mask,       "klong", limits = [0, 0.1])


In [ ]:
# Loading the different inputs
BSD_dataset = BlackSea_Dataset(year_start  = 2010,
                               year_end    = 2020,
                               month_start = 1,
                               month_end   = 12)

data_oxygen        = BSD_dataset.get_data(variable = "oxygen")
mask               = BSD_dataset.get_mask(False)
maskCS             = BSD_dataset.get_mask(True)

# Extracting the training set
training_set = data_oxygen[: 365 * 6]

# Computing the mean
mean_6years = np.mean(training_set, axis = 0)

# Stores the mean each year
mean_each_year = [np.mean(data_oxygen[i * 365 : (i + 1) * 365], axis = 0) for i in range(6)]

def save_plot(data : np.array, mask : np.array, title : str):
    """Save the plot of the mean"""
    plt.figure(figsize = (12, 8))
    plt.imshow(data, cmap='viridis', vmin = 0, vmax = 1)
    plt.colorbar(fraction = 0.021)
    plt.imshow(mask, cmap='gray', alpha = 0.025)
    date_string = f"Mean : {title}"
    ax = plt.gca()
    box = Rectangle((0.80, 0.90), 0.56, 0.16, edgecolor='black', facecolor='black', transform=ax.transAxes)
    plt.annotate(date_string, xy=(0.98, 0.965), xycoords='axes fraction',
                horizontalalignment='right', verticalalignment='top',
                fontsize=12, color='white')
    ax.add_patch(box)
    plt.grid(True, alpha=0.1)
    plt.savefig(f'../../analysis/means/means_{title}.png')

# Saving the plot for all the years
save_plot(mean_6years, maskCS, "Training")

# Saving the plot for each year
for i, n in enumerate(["2010", "2011", "2012", "2013", "2014", "2015"]):
    save_plot(mean_each_year[i], maskCS, n)

# Normalized deoxygenation treshold
hypox_tresh = xarray.open_dataset(BSD_dataset.paths[0])["HYPON"].data.item()

# Computing the average state of the region (Hypoxia or not)
mean_6years_state = mean_6years < hypox_tresh

# Stores the mean each year
mean_each_year_state = [mean < hypox_tresh for mean in mean_each_year]

def save_plot_state(data : np.array, mask : np.array, title : str):
    """Save the plot of the mean"""
    plt.figure(figsize = (12, 8))
    plt.imshow(data, cmap='viridis', vmin = 0, vmax = 1)
    cbar = plt.colorbar(fraction=0.021)
    cbar.set_ticks([0, 1])
    cbar.set_ticklabels(['Oxygenated', 'Hypoxia'])
    plt.imshow(mask, cmap='gray', alpha = 0.25)
    date_string = f"Mean : {title}"
    ax = plt.gca()
    box = Rectangle((0.80, 0.90), 0.56, 0.16, edgecolor='black', facecolor='black', transform=ax.transAxes)
    plt.annotate(date_string, xy=(0.98, 0.965), xycoords='axes fraction',
                horizontalalignment='right', verticalalignment='top',
                fontsize=12, color='white')
    ax.add_patch(box)
    plt.grid(True, alpha=0.1)
    plt.savefig(f'../../analysis/means/means_{title}.png')

# Saving the plot for all the years
save_plot_state(mean_6years_state, maskCS, "Training (H)")

# Saving the plot for each year
for i, n in enumerate(["2010 (H)", "2011 (H)", "2012 (H)", "2013 (H)", "2014 (H)", "2015 (H)"]):
    save_plot_state(mean_each_year_state[i], maskCS, n)


In [ ]:
# Sending everything to wandb
wandb.init(project = "ESA - Repport")

# Sending the animations
wandb.log({"Oxygen":              wandb.Video("../../analysis/images/oxygen.gif", fps = 1),
           "Temperature":         wandb.Video("../../analysis/images/temperature.gif", fps = 1),
           "Salinity":            wandb.Video("../../analysis/images/salinity.gif", fps = 1),
           "Chlorophyll":         wandb.Video("../../analysis/images/chlorophyll.gif", fps = 1),
           "Reflectance (Short)": wandb.Video("../../analysis/images/kshort.gif", fps = 1),
           "Reflectance (Long)" : wandb.Video("../../analysis/images/klong.gif", fps = 1)})

# Sending the mean concentrations
wandb.log({"Concentration (Mean, 2010-2015)": wandb.Image(f"../../analysis/means/means_Training.png"),
           "Concentration (Mean, 2010)":      wandb.Image(f"../../analysis/means/means_2010.png"),
           "Concentration (Mean, 2011)":      wandb.Image(f"../../analysis/means/means_2011.png"),
           "Concentration (Mean, 2012)":      wandb.Image(f"../../analysis/means/means_2012.png"),
           "Concentration (Mean, 2013)":      wandb.Image(f"../../analysis/means/means_2013.png"),
           "Concentration (Mean, 2014)":      wandb.Image(f"../../analysis/means/means_2014.png"),
           "Concentration (Mean, 2015)":      wandb.Image(f"../../analysis/means/means_2015.png")})

# Sending the mean states
wandb.log({"State (Mean, 2010-2015)": wandb.Image(f"../../analysis/means/means_Training_(H).png"),
           "State (Mean, 2010)":      wandb.Image(f"../../analysis/means/means_2010_(H).png"),
           "State (Mean, 2011)":      wandb.Image(f"../../analysis/means/means_2011_(H).png"),
           "State (Mean, 2012)":      wandb.Image(f"../../analysis/means/means_2012_(H).png"),
           "State (Mean, 2013)":      wandb.Image(f"../../analysis/means/means_2013_(H).png"),
           "State (Mean, 2014)":      wandb.Image(f"../../analysis/means/means_2014_(H).png"),
           "State (Mean, 2015)":      wandb.Image(f"../../analysis/means/means_2015_(H).png")})

# Closing the wandb session
wandb.finish()


<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Playground</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
# -----------------
#    Parameters
# -----------------
#
# Time window
month_starting = 1
month_ending   = 2
year_starting  = 1980
year_ending    = 1980

# ------------------
#  Loading the data
# ------------------
# Loading the different datasets
BSD_dataset = BlackSea_Dataset(year_start  = year_starting,
                               year_end    = year_ending,
                               month_start = month_starting,
                               month_end   = month_ending)

# Loading the days ID (used to give temporal information to the model)
days_ID = BSD_dataset.get_days()

# Loading the different inputs
data_temperature   = BSD_dataset.get_data(variable = "temperature")
data_salinity      = BSD_dataset.get_data(variable = "salinity")
data_chlorophyll   = BSD_dataset.get_data(variable = "chlorophyll")
data_kshort        = BSD_dataset.get_data(variable = "kshort")
data_klong         = BSD_dataset.get_data(variable = "klong")

# Loading the output
data_oxygen = BSD_dataset.get_data(variable = "oxygen")

# Loading spatial information
bathy = BSD_dataset.get_depth(unit = "meter")
mesh  = BSD_dataset.get_mesh(x = 256, y = 576)

# Hypoxia treshold
hypox_tresh = xarray.open_dataset(BSD_dataset.paths[0])["HYPON"].data.item()

# Loading the black sea masks
bs_mask             = BSD_dataset.get_mask(continental_shelf = False)
bs_mask_with_depth  = BSD_dataset.get_mask(continental_shelf = True)
bs_mask_complete    = get_complete_mask(data_oxygen, hypox_tresh, bs_mask_with_depth)


# -----------------------
#  Preprocessing the data
# -----------------------
#
# Creation of the dataloader
BSD_loader = BlackSea_Dataloader(x = [data_temperature],
                                 y = data_oxygen,
                                 t = days_ID,
                              mesh = mesh,
                              mask = bs_mask,
                   mask_with_depth = bs_mask_with_depth,
                        bathymetry = bathy,
                        window_inp = 1,
                        window_out = 1,
                    window_transfo = 1,
                              mode = "regression",
                  hypoxia_treshold = hypox_tresh)

# Retrieving the datasets
ds_train      = BSD_loader.get_dataloader("train")
ds_validation = BSD_loader.get_dataloader("validation")
ds_test       = BSD_loader.get_dataloader("test")

# Extracting the normalized oxygen treshold value
norm_oxy = BSD_loader.get_normalized_deoxygenation_treshold()


In [ ]:
for x, t, y in ds_train:

    print(x.shape)

    break

In [ ]:
# -------------------------------------------------------
#
#        |
#       / \
#      / _ \                  ESA - PROJECT
#     |.o '.|
#     |'._.'|          BLACK SEA DEOXYGENATION EMULATOR
#     |     |
#   ,'|  |  |`.             BY VICTOR MANGELEER
#  /  |  |  |  \
#  |,-'--|--'-.|                2023-2024
#
#
# -------------------------------------------------------
#
# Documentation
# -------------
# A neural network definition to be used as temporal encoder
#
# Pytorch
import torch.nn as nn


class ENCODER(nn.Sequential):
    r"""A neural network used to encode the temporal information of the data and return weights for the input data"""

    def __init__(self, input_size : int):
        super(ENCODER, self).__init__()

        # Defining the layers
        self.linear_in       = nn.Linear(input_size, 256)
        self.linear_middle_1 = nn.Linear(256,        256)
        self.linear_middle_2 = nn.Linear(256,        128)
        self.linear_middle_3 = nn.Linear(128,         64)
        self.linear_middle_4 = nn.Linear(64,          32)
        self.linear_out      = nn.Linear(32,           1)

        # Defining the activation functions
        self.activation = nn.GELU()

        # Defining the softmax function, i.e. (t, values, day) to (t, values, 1) then (t, weights, 1)
        self.softmax = nn.Softmax(dim = 0)

    def forward(self, x):

        # Applying the layers
        x = self.activation(self.linear_in(x))
        x = self.activation(self.linear_middle_1(x))
        x = self.activation(self.linear_middle_2(x))
        x = self.activation(self.linear_middle_3(x))
        x = self.activation(self.linear_middle_4(x))
        x = self.linear_out(x)

        # Applying the softmax function
        return self.softmax(x)

    def count_parameters(self,):
        r"""Determines the number of trainable parameters in the model"""
        return int(sum(p.numel() for p in self.parameters() if p.requires_grad))

class FCNN(nn.Sequential):
    r"""A fully convolutional neural network"""

    def __init__(self, problem: str, inputs: int, outputs: int, window_transformation: int = 1, kernel_size : int = 3, scaling : int = 1):
        super(FCNN, self).__init__()

        # Initialization
        self.n_in    = inputs
        self.problem = problem
        self.padding = kernel_size // 2

        # Number of output channels, i.e. times 2 because either mean and std for regression or both classes for classification
        self.n_out   = outputs * 2

        # ------ Architecture ------
        #
        # Temporal Encoder
        self.block_encoder = ENCODER(window_transformation)

        # Main Layers
        self.conv_init           = nn.Conv2d(self.n_in    , 256 * scaling, kernel_size, padding = self.padding)
        self.conv_intermediate_1 = nn.Conv2d(256 * scaling, 128 * scaling, kernel_size, padding = self.padding)
        self.conv_intermediate_2 = nn.Conv2d(128 * scaling,  64 * scaling, kernel_size, padding = self.padding)
        self.conv_intermediate_3 = nn.Conv2d( 64 * scaling,  32 * scaling, kernel_size, padding = self.padding)
        self.conv_final          = nn.Conv2d( 32 * scaling,    self.n_out, kernel_size, padding = self.padding)

        # Activation function
        self.activation = nn.GELU()

        # Normalization
        self.normalization_init           = nn.BatchNorm2d(self.conv_init.out_channels)
        self.normalization_intermediate_1 = nn.BatchNorm2d(self.conv_intermediate_1.out_channels)
        self.normalization_intermediate_2 = nn.BatchNorm2d(self.conv_intermediate_2.out_channels)
        self.normalization_intermediate_3 = nn.BatchNorm2d(self.conv_intermediate_3.out_channels)

    def forward(self, x, t):

        # Retrieiving dimensions (Ease of comprehension)
        samples, days, values, variables, x_res, y_res = x.shape

        # ----- Encoding Time -----
        #
        # Applying the encoder
        weights = torch.squeeze(self.block_encoder(t), dim = -1)

        # Applying the weights (except to mesh (dim = 2) and bathymetry (dim = 3))
        for sample in range(samples):
            for value in range(days):
                x[:, value, :, :-3] *= weights[sample, value]

        # Reshaping
        x = x.reshape(samples, days * values * variables, x_res, y_res)

        # ----- Fully Convolutionnal -----
        #
        x = self.normalization_init(self.activation(self.conv_init(x)))
        x = self.normalization_intermediate_1(self.activation(self.conv_intermediate_1(x)))
        x = self.normalization_intermediate_2(self.activation(self.conv_intermediate_2(x)))
        x = self.normalization_intermediate_3(self.activation(self.conv_intermediate_3(x)))
        x = self.conv_final(x)

        # Retrieiving dimensions (Ease of comprehension)
        b, c, x_res, y_res = x.shape

        # Reshaping the output, i.e. (samples, days, values, x, y)
        return x.reshape(b, self.n_out // 2, 2, x_res, y_res)

    def count_parameters(self,):
        r"""Determines the number of trainable parameters in the model"""
        return int(sum(p.numel() for p in self.parameters() if p.requires_grad))

class AVERAGE(nn.Sequential):
    r"""A 'neural network' that predicts the pixel temporal average (should be used a baseline)"""

    def __init__(self, data_output : np.array, device : str, kwargs : dict):
        super(AVERAGE, self).__init__()

        # Extracting information
        dataset_size     = [0.6, 0.3]
        problem          = "regression"
        hypoxia_treshold = 0.1

        # Retrieiving dimensions
        t, x, y = data_output.shape

        # Number of training samples
        train_samples = int(t * dataset_size[0])

        # ----- Regression ------
        if problem == "regression":

            # Determine the minimum and maximum values of the data
            min_value = np.nanmin(data_output)
            max_value = np.nanmax(data_output)

            # Determining the minimum and maximum values
            min_value = np.nanmin(data_output)
            max_value = np.nanmax(data_output)

            # Shift the data to ensure minimum value is 0
            shifted_data = data_output - min_value

            # Normalizing the data
            normalized_data = shifted_data / (max_value - min_value)

            # Predicting the average and log of variance
            average_output = torch.mean(torch.from_numpy(normalized_data[: train_samples, :, :]), dim = 0)
            std_output     = torch.log(torch.var(torch.from_numpy(normalized_data[: train_samples, :, :]), dim = 0))

            # Stacking
            average_output = torch.stack([average_output, std_output])

        # ----- Classification ------
        else:

            # Converting to classification
            average_output = torch.from_numpy((data_output[: train_samples, :, :] < hypoxia_treshold) * 1)

            # Summing over time, i.e. if total number of hypoxic days is greater than 50% of the time, then it is hypoxic
            average_output = (torch.sum(average_output, dim = 0) > train_samples // 2) * 1

            # Conversion to "probabilities", i.e. (t, x, y) to (t, c, x, y) with c = 0 no hypoxia, c = 1 hypoxia
            average_output = torch.stack([(average_output == 0) * 1, average_output]).float()

        # Storing information
        self.outputs = 1
        self.bs      = 64
        self.average = self.process(average_output)
        self.device  = "cuda"

        # Dummy feature (It plays no role whatsoever, it is just a placeholder to make the model work with the trainer)
        self.layer = nn.Conv2d(1, 1, 1)

    def forward(self, x, t):
        return to_device(self.average[:x.shape[0]], self.device)

    def process(self, x : torch.Tensor):
        r"""Used to format the output to the correct shape"""

        # Adding number of forecasted days
        x = torch.unsqueeze(x, dim = 0) if self.outputs == 1 else \
            torch.stack([x for i in range(self.outputs)], dim = 0)

        # Adding batch size
        return torch.stack([x for i in range(self.bs)], dim = 0)

    def count_parameters(self,):
        r"""Determines the number of trainable parameters in the model"""
        return int(0)

In [ ]:
# Neural Network stuff
neural_network = FCNN(problem = "regression",
                      inputs  = 4,
                      outputs = 1,
                      window_transformation = 1,
                      kernel_size = 1,
                      scaling = 1)


#neural_network = AVERAGE(data_oxygen, "cuda", {})


optimizer      = optim.Adam(neural_network.parameters(), lr = 0.001)

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

neural_network.to(device)

In [ ]:
get_ratios_plot(data_oxygen, hypox_tresh, bs_mask_with_depth)

In [ ]:
# ------------------------------------------------
show = False

for epoch in range(1):

    metrics_tool = BlackSea_Metrics(mode = "regression",
                                    mask = bs_mask_with_depth,
                           mask_complete = bs_mask_complete,
                                treshold = norm_oxy,
                       number_of_samples = BSD_loader.get_number_of_samples("validation"))

    for x, t, y in ds_train:

        x, t, y = x.to(device), t.to(device), y.to(device)
        prediction = neural_network(x, t)
        loss_training = compute_loss(y_pred = prediction, y_true = y,mask = bs_mask_with_depth, problem = "regression", device = "cpu", kwargs = {})
        print(f"E{epoch} - Loss (Training):", loss_training.item())
        optimizer.zero_grad()
        loss_training.backward()
        optimizer.step()

        # Cleaning
        del x, t, y, prediction, loss_training
        torch.cuda.empty_cache()
        break


    with torch.no_grad():

        # Stores all the predictions for the metrics (plots)
        prediction_all = None

        for x, t, y in ds_validation:

            # Making prediction
            x, t, y = x.to(device), t.to(device), y.to(device)
            prediction = neural_network(x, t)
            loss_validation = compute_loss(y_pred = prediction, y_true = y, mask = bs_mask_with_depth, problem = "regression", device = "cpu", kwargs = {})
            print(f"E{epoch} - Loss (Validation):", loss_validation.item())
            x, t, y, prediction = x.to("cpu"), t.to("cpu"), y.to("cpu"), prediction.to("cpu")

            """
            # Plotting mean against ground truth in a subplot
            if show:

                # Highlighting hypoxic areas
                y_hyp = ( y < norm_oxy ) * 1.0
                p_hyp = ( prediction < norm_oxy ) * 1.0

                # Hiding non-obserable areas
                p_hyp[:,:,:, y[0, 0, 0] == -1] = torch.nan
                y_hyp[:,:,:, y[0, 0, 0] == -1] = torch.nan


                plt.figure(figsize = (20, 20))
                plt.subplot(1, 3, 1)
                plt.imshow(torch.flipud(y_hyp[0, 0, 0]))
                plt.subplot(1, 3, 2)
                plt.imshow(torch.flipud(p_hyp[0, 0, 0]))
                plt.subplot(1, 3, 3)
                plt.imshow(torch.flipud(y_hyp[0, 0, 0]) - torch.flipud(p_hyp[0, 0, 0]))
                plt.setp(plt.gcf().get_axes(), xticks = [], yticks = [])
                plt.subplot(1, 3, 1).set_title("Ground Truth", fontsize = 6)
                plt.subplot(1, 3, 2).set_title("Prediction", fontsize = 6)
                plt.subplot(1, 3, 3).set_title("Difference", fontsize = 6)
                plt.show()

                prediction[:,:,:, y[0,0,0,:,:] == -1] = torch.nan
                y[:,:,:, y[0,0,0,:,:] == -1] = torch.nan

                plt.figure(figsize = (20, 20))
                plt.subplot(1, 3, 1)
                plt.imshow(torch.flipud(y[0, 0, 0]),  vmin = 0, vmax = 1)
                plt.subplot(1, 3, 2)
                plt.imshow(torch.flipud(prediction[0, 0, 0]), vmin = 0, vmax = 1)
                plt.subplot(1, 3, 3)
                plt.imshow(torch.flipud(torch.exp(prediction[0, 0, 1]/2)), vmin = 0, vmax = 1)
                plt.setp(plt.gcf().get_axes(), xticks = [], yticks = [])
                plt.subplot(1, 3, 1).set_title("Ground Truth", fontsize = 6)
                plt.subplot(1, 3, 2).set_title("Prediction (Mean)", fontsize = 6)
                plt.subplot(1, 3, 3).set_title("Prediction (Std)", fontsize = 6)
                plt.show()

            """

            # Concatenating all the predictions
            prediction_all = torch.cat((prediction_all, prediction), dim = 0) if prediction_all is not None else prediction

            del x, t, y, prediction
            torch.cuda.empty_cache()


    # Sampling random data for comparison
    # Metrics
    y_vall_all = torch.from_numpy(BSD_loader.y_validation)
    #metrics_tool.compute_metrics(y_pred = prediction_all, y_true = y_vall_all)
    metrics_tool.compute_plots_comparison_regression(y_pred = prediction_all, y_true = y_vall_all)


    #metrics_tool.compute_plots(  y_pred = prediction_all, y_true = y_vall_all)

    # Getting the results
    """
    if show:
        results, results_name = metrics_tool.get_results()
        for r, n in zip(results[0], results_name):
            print(n, " : ", r)
        print("\n")
    """


In [ ]:
BSD_loader.get_number_of_samples("validation")

In [ ]:

a = np.random.randint(0, prediction_all.shape[0] - 1, 3)
print(a)

In [ ]:
for i in a:
    print(i)